In [23]:
import newspaper

_20m_paper = newspaper.build('https://www.20minutes.fr/politique/', memoize_articles=False,fetch_images=False)

papers = []
urls_set = set()
for article in _20m_paper.articles:
   # check to see if the article url is not within the urls_set
   if article.url not in urls_set:
     # add the unique article url to the set
     urls_set.add(article.url)
     # remove all links for article commentaires
     if not str(article.url).endswith('commentaires'):
        papers.append(article.url)

print(len(papers)) 

176


In [24]:
import newspaper
import requests
from newspaper.utils import BeautifulSoup

papers = []
urls_set = set()

base_url = 'https://www.20minutes.fr/'

_20m_paper = newspaper.build(base_url, 
fetch_images=False, memoize_articles=False)

for article in _20m_paper.articles:
   # check to see if the article url is not within the urls_set
   if article.url not in urls_set:
     # add the unique article url to the set
     urls_set.add(article.url)
     # remove all links for article commentaires
     if not str(article.url).endswith('commentaires'):
        papers.append(article.url)

 
_20_urls = {'societe': base_url+'societe',
             'politique': base_url+'politique',
             'faits_divers': base_url+'faits_divers',
             'monde': base_url+'monde',
             'culture':base_url+'culture',
             'people': base_url+'people',
             'sports': base_url+'sports',
             'economie': base_url+'economie',
             'sciences': base_url+'sciences',
             'planete': base_url+'planete',
             'high-tech' : base_url+'high-tech',
             }


for category, url in _20_urls.items():
   raw_html = requests.get(url)
   soup = BeautifulSoup(raw_html.text, 'html.parser')
   for articles_tags in soup.findAll('div', {'class': 'articles'}):
      for article_href in articles_tags.find_all('a', href=True):
         if not str(article_href['href']).endswith('commentaires'):
           urls_set.add(article_href['href'])
           papers.append(article_href['href'])

print(len(papers))

176


In [90]:
import time
import requests
from bs4 import BeautifulSoup
import json
import string
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

papers = []
urls_set = set()

browser = webdriver.Chrome('/mnt/c/users/tdenimal/Downloads/chromedriver_win32/chromedriver.exe')
wait = WebDriverWait(browser, 10)
browser.get(base_url+'societe')
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")


while True:
    try:
        time.sleep(1)
        show_more = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@type="button"][contains(.,"Voir plus")]')))
        browser.execute_script("arguments[0].click();", show_more)
        print(show_more)
    except Exception as e:
            print(e)
            break
    
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    for articles_tags in soup.findAll('article'):
        for article_href in articles_tags.find_all('a', href=True):
            if not str(article_href['href']).endswith('commentaires'):
                urls_set.add(article_href['href'])
                papers.append(article_href['href'])

    print(len(papers))

<selenium.webdriver.remote.webelement.WebElement (session="4dfd66bd1c0371674d7606f68c80d58a", element="d3af8ec8-d4f4-4d73-93cb-68ce045c5e61")>
30
<selenium.webdriver.remote.webelement.WebElement (session="4dfd66bd1c0371674d7606f68c80d58a", element="d3af8ec8-d4f4-4d73-93cb-68ce045c5e61")>
70
<selenium.webdriver.remote.webelement.WebElement (session="4dfd66bd1c0371674d7606f68c80d58a", element="d3af8ec8-d4f4-4d73-93cb-68ce045c5e61")>
120
<selenium.webdriver.remote.webelement.WebElement (session="4dfd66bd1c0371674d7606f68c80d58a", element="d3af8ec8-d4f4-4d73-93cb-68ce045c5e61")>
180
<selenium.webdriver.remote.webelement.WebElement (session="4dfd66bd1c0371674d7606f68c80d58a", element="d3af8ec8-d4f4-4d73-93cb-68ce045c5e61")>
250
<selenium.webdriver.remote.webelement.WebElement (session="4dfd66bd1c0371674d7606f68c80d58a", element="d3af8ec8-d4f4-4d73-93cb-68ce045c5e61")>
330
<selenium.webdriver.remote.webelement.WebElement (session="4dfd66bd1c0371674d7606f68c80d58a", element="d3af8ec8-d4f4-4d7

KeyboardInterrupt: 

In [131]:
base_url

'https://www.20minutes.fr/'

In [129]:
from newspaper import Article
first_article = Article(url=base_url+papers[1][1:], language='fr')
first_article.download()
first_article.parse()
first_article.text


ArticleException: Article `download()` failed with HTTPSConnectionPool(host='www.20minutes.fr', port=443): Max retries exceeded with url: /societe/2872907-20200929-attaque-paris-scenario-agression-couteau-devant-charlie-hebdo-precise (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f3415e1f710>: Failed to establish a new connection: [Errno -2] Name or service not known'))) on URL https://www.20minutes.fr/societe/2872907-20200929-attaque-paris-scenario-agression-couteau-devant-charlie-hebdo-precise

In [112]:
import spacy
from spacy import displacy
from collections import Counter

In [125]:
import fr_core_news_lg

In [126]:
nlp = fr_core_news_lg.load()

In [122]:
doc = nlp(first_article.text)
print([(X.text, X.label_) for X in doc.ents])

[('VILLEFONTAINE', 'ORG'), ('Grenoble', 'LOC'), ('Villefontaine', 'PER'), ('Victorine', 'LOC'), ('— A. Merlet', 'PER'), ('AFP', 'ORG'), ('Victorine', 'PER'), ('Villefontaine', 'LOC'), ('Isère', 'LOC'), ('étang de Saint-Bonnet', 'LOC'), ('Villefontaine', 'LOC'), ('Isère', 'LOC'), ('Victorine', 'LOC'), ('Vienne', 'LOC'), ('république de Grenoble', 'LOC'), ('Victorine DARTOIS', 'MISC'), ('St Bonnet', 'LOC'), ('Villefontaine', 'LOC'), ('👮', 'MISC'), ("Gendarmerie de l'Isère", 'ORG'), ('September', 'MISC'), ('Victorine', 'PER'), ('»', 'MISC'), ('Audrey Quey', 'PER'), ('République de Vienne', 'LOC'), ('Grenoble', 'LOC'), ('» du décès de la jeune femme', 'MISC'), ('»', 'MISC'), ('Le Dauphiné Libéré', 'MISC'), ('Saint-Hubert', 'PER'), ('Victorine', 'PER'), ('Marques', 'LOC'), ('Villefontaine', 'LOC'), ('Fougères', 'LOC'), ('Victorine', 'LOC'), ('Villefontaine', 'LOC'), ('Victorine', 'LOC')]


In [127]:
doc = nlp(first_article.text)
print([(X.text, X.label_) for X in doc.ents])

[('VILLEFONTAINE', 'ORG'), ('Grenoble', 'LOC'), ('Villefontaine', 'LOC'), ('Victorine', 'MISC'), ('A. Merlet', 'PER'), ('AFP', 'ORG'), ('Victorine', 'MISC'), ('Villefontaine', 'LOC'), ('Isère', 'LOC'), ('réserve naturelle de l’étang de Saint-Bonnet', 'LOC'), ('Villefontaine', 'LOC'), ('Isère', 'LOC'), ('Victorine', 'LOC'), ('Vienne', 'LOC'), ('république de Grenoble', 'LOC'), ('20 Minutes', 'MISC'), ('Victorine DARTOIS', 'PER'), ('étang de St Bonnet', 'LOC'), ('19H', 'LOC'), ('Villefontaine', 'LOC'), ('👮', 'MISC'), ("Gendarmerie de l'Isère", 'ORG'), ('September 28', 'MISC'), ('Victorine', 'MISC'), ('Audrey Quey', 'PER'), ('République de Vienne', 'LOC'), ('Grenoble', 'LOC'), ('Le Dauphiné Libéré', 'ORG'), ('Saint-Hubert', 'PER'), ('Victorine', 'PER'), ('Marques', 'LOC'), ('Villefontaine', 'LOC'), ('Fougères', 'LOC'), ('Victorine', 'LOC'), ('Villefontaine', 'LOC'), ('Victorine', 'PER')]
